# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("homepage_actions.csv")
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [4]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [14]:
df.value_counts("id")

id
937217    2
575502    2
583751    2
583515    2
583153    2
         ..
568845    1
568937    1
568992    1
569142    1
182988    1
Length: 6328, dtype: int64

In [8]:
#So I made the mistake below of trying to drop the people who did the same
#thing twice, but from that mistake I learned that nobody actually did
df.drop_duplicates(subset = ["id", "action"], inplace = True)

In [9]:
df.action.value_counts()
#these are same as before

view     6328
click    1860
Name: action, dtype: int64

In [28]:
multis = df[df["id"].isin(list(df['id'].value_counts()[df['id'].value_counts()>1].index))]

,timestamp,id,group,action
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
11,2016-09-24 21:05:15.348935,601714,experiment,view
12,2016-09-24 21:06:27.553057,601714,experiment,click
14,2016-09-24 21:29:19.766467,487634,experiment,view
...,...,...,...,...
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8181,2017-01-18 09:07:37.661143,795585,control,view


In [29]:
# now... is it that every repeat customer clicked on their second visit?
# Those people should count as clicks overall
firsts = multis.drop_duplicates(subset = "id", keep = "first")
seconds = multis.drop_duplicates(subset = "id", keep = "last")

In [30]:
firsts.value_counts("action")

action
view    1860
dtype: int64

In [31]:
seconds.value_counts("action")

action
click    1860
dtype: int64

In [32]:
#yep. so every one of these guys should count as a click. 
#I'll delete all of their first entries to reflect that

df.drop_duplicates(subset = "id", keep = "last", inplace = True)

<bound method DataFrame.value_counts of                        timestamp      id       group action
0     2016-09-24 17:42:27.839496  804196  experiment   view
1     2016-09-24 19:19:03.542569  434745  experiment   view
2     2016-09-24 19:36:00.944135  507599  experiment   view
3     2016-09-24 19:59:02.646620  671993     control   view
4     2016-09-24 20:26:14.466886  536734  experiment   view
...                          ...     ...         ...    ...
8183  2017-01-18 09:11:41.984113  192060  experiment   view
8184  2017-01-18 09:42:12.844575  755912  experiment   view
8185  2017-01-18 10:01:09.026482  458115  experiment   view
8186  2017-01-18 10:08:51.588469  505451     control   view
8187  2017-01-18 10:24:08.629327  461199     control   view

[6328 rows x 4 columns]>

In [33]:
df.value_counts("action")

action
view     4468
click    1860
dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [36]:
df["value"] = 0
df.loc[df["action"] == "click", "value"] = 1

In [39]:
df.value_counts("value")

value
0    4468
1    1860
dtype: int64

In [41]:
exps = df[df["group"] == "experiment"]
conts = df[df["group"] == "control"]

In [42]:
stats.ttest_ind(exps["value"], conts["value"])

Ttest_indResult(statistic=2.6195696844542202, pvalue=0.008825098914958293)

In [ ]:
#That's a little pvalue.
#Suddenly, the null hypothesis and I have one thing in common

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [43]:
c_rate = conts.value.mean()
expect_clicks_for_experi = c_rate*len(exps)
print(expect_clicks_for_experi)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [48]:
clicks_exp = np.sum(exps["value"])
clicks_exp

928

In [51]:
n = len(exps)
p = c_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [52]:
stds_away_from_expect = (clicks_exp - expect_clicks_for_experi)/(std)
stds_away_from_expect

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [55]:
stats.norm.sf((stds_away_from_expect))

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Well... sorta? They both reject the null but the stats.norm.sf version gave me a very different p value.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.